# A Curious Course on Coroutines and Concurrency 

## Disclaimers

- All content of this jupyter notebook are referred to David Beazley's [slide](http://117.128.6.29/cache/www.dabeaz.com/coroutines/Coroutines.pdf?ich_args2=526-03152400039188_616107d40934da0c48d67a98c79844af_10001002_9c896228dfc2f6d19f32518939a83798_460d1e2e54e6fd2148466980ccb734cb).


- I made some revisions in order to run correctly.


- Please run in Python3.

## Part １ Introduction to Generators and Coroutines

### 1.1 Generators

   - A generator is a function that produces a sequence of results instead of a single value

   - Instead of returning a value, you generate a series of values (using the yield statement)

   - Typically, you hook it up to a for-loop

In [1]:
def countdown(n):
    while n > 0:
        yield n
        n -= 1

In [2]:
for i in countdown(5):
    print(i)

5
4
3
2
1


### 1.2 Generators
　 
   - Behavior is quite different than normal func
   
   - Calling a generator function creates an generator object. However, it does **not** start running the function.

In [3]:
def countdown(n):
    print("Counting down from", n)
    while n > 0:
        yield n
        n -= 1

In [4]:
# Note that no output was produced
x = countdown(5)

In [5]:
x

<generator object countdown at 0x7f950c4f3e08>

### 1.3. Generator Functions

- The function only executes on next()

- yield produces a value, but suspends the function

- Function resumes on next call to next()

- When the generator returns, iteration stops

In [6]:
x = countdown(3)

In [7]:
x

<generator object countdown at 0x7f950c4f39e8>

In [8]:
# Function starts executing here
next(x)

Counting down from 3


3

In [9]:
next(x)

2

In [10]:
next(x)

1

In [11]:
next(x)

StopIteration: 

### 1.4. A Practical Example

In [12]:
# A Python version of Unix 'tail -f'

import time

def follow(thefile):
    # thefile.seek(0, 2)
    while True:
        line = thefile.readline()
        if not line:
            break
            # time.sleep(0.1)
            # continue
        yield line

In [13]:
!echo "It is summer day\nI am studying Coroutine in Python\nHi, ladies and gentlemen" > access-log

In [14]:
!cat access-log

It is summer day
I am studying Coroutine in Python
Hi, ladies and gentlemen


In [15]:
with open("access-log", "r") as f:
    for line in follow(f):
        print(line)

It is summer day

I am studying Coroutine in Python

Hi, ladies and gentlemen



### 1.5. Generators as Pipelines

- One of the most powerful applications of generators is setting up processing pipelines


- Similar to shell pipes in Unix
  
  input sequeence --> generator1 --> generator2 --> generator3
  --> for x in s:
  
  
- Idea: You can stack a series of generator functions together into a pipe and pull items through it with a for-loop
  

In [16]:
# A Pipeline Example
# Print all server log entries containing 'Python'

def grep(pattern, lines):
    for line in lines:
        if pattern in line:
            yield line

# Set up a processing pipe: tail -f | grep python
with open("access-log", "r") as logfile:
    loglines = follow(logfile)
    greplines = grep("Python", loglines)
    for line in greplines:
        print(line)

I am studying Coroutine in Python



### 1.6. Yield as an Expression 

- In Python 2.5, a slight modification to the yield　statement was introduced (PEP-342)


- You could now use yield as an *expression*


- For example, on the right side of an assignment

In [17]:
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

### 1.7. Coroutines


- If If you use yield more generally, you get a coroutine


- These do more than just generate values


- Instead, functions can consume values **sent** to it


- Sent values are returned by (yield)

In [18]:
g = grep("python")  # Notice that no output was produced

In [19]:
g

<generator object grep at 0x7f950c4f38e0>

In [20]:
next(g)  # On first operation, coroutine starts running

Looking for python


In [21]:
g.send("Hi, I am Summer")

In [23]:
g.send("I am studying python now")

I am studying python now


### 1.8. Coroutine Execution

- Execution is the same ss for a generator


- When you call a coroutine, nothing happens


- They only run in response to next() and send() methods

```
# Notice that no output was produced
>> g = grep("python")
#  On first operation, coroutine starts running
>> next(g)
Looking for python
```

### 1.9. Coroutine Priming

- All coroutines must be "primed" by first calling .next() (or send(None))


- This advances execution to the location of the first yield expression.

```
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        # next() advances the coroutine to 
        # the first yield expression (yield) 
        line = (yield)  
        if pattern in line:
            print(line)
```

- At this point, it's ready to receive a value

### 1.10. Using a Decorator


- Remembering to call .next() is easy to forget


- Solved by wrapping coroutines with a decorator

In [24]:
def coroutine(func):
    def start(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr)
        return cr
    return start

@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    while True:
        line = (yield)
        if pattern in line:
            print(line)

In [25]:
g = grep("python")

Looking for python


In [26]:
g.send("Hi, I am Summer")

In [27]:
g.send("I am studing coroutine in python")

I am studing coroutine in python


### 1.11. Closing a Coroutine

- A coroutine might run indefinitely


- Use .close() to shut it down


- Note: Garbage collection also calls close()

In [28]:
g = grep("python")

Looking for python


In [29]:
g.send("Hi")

In [30]:
g.send("coroutine in python")

coroutine in python


In [31]:
g.close()

In [32]:
g

<generator object grep at 0x7f950c4492b0>

In [33]:
g.send("hi")

StopIteration: 

### 1.12. Catching close()


- close() can be caught (GeneratorExit)


- You cannot ignore this exception


- Only legal action is to clean up and return

In [34]:
@coroutine
def grep(pattern):
    print("Looking for %s" % pattern)
    try:
        while True:
            line = (yield)
            if pattern in line:
                print(line)
    except GeneratorExit:
        print("Going away. Goodbye")

In [35]:
g = grep("python")

Looking for python


In [36]:
g.send("python")

python


In [37]:
g.close()

Going away. Goodbye


### 1.13. Throwing an Exception

- Exceptions can be thrown inside a coroutine


- Exception originates at the yield expression


- Can be caught / handled in the usual ways

In [38]:
g = grep("python")

Looking for python


In [39]:
g.send("python rocks")

python rocks


In [40]:
g.throw(RuntimeError, "You are hosed")

RuntimeError: You are hosed

### 1.14. **Interlude**

- Despite some similarities, Generators and coroutines are basically two different concepts


- Generators produce values


- Coroutines tend to consume values


- It is easy to get sidetracked because methods meant for coroutines are sometimes described as a way to tweak generators that are in the process of producing an iteration pattern (i.e., resetting its value). This is mostly bogus.

### 1.15. A Bogus Example


- A "generator" that produces **and** receives values

In [41]:
def countdown(n):
    print("Count down from %d \n" % n)
    while n >= 0:
        print("Into while")
        newvalue = (yield n)
        print("newvalue is", newvalue)
        # if a new value got sent in, reset n with it
        if newvalue is not None:
            print("Reset n=%d to n=%d\n" % (n, newvalue))
            n = newvalue
            # print(n)
        else:
            print("n -= 1\n")
            n -= 1

- It runs, but it's "flaky" and hard to understand
  
  **Notice how a value got "lost" in the iteration protocol**
   

In [42]:
c = countdown(6)
for n in c:
    print("Into for %d" % n)
    if n == 5:
        print("!!! Into send !!!")
        c.send(3)
        print("!!! Out send !!!")


Count down from 6 

Into while
Into for 6
newvalue is None
n -= 1

Into while
Into for 5
!!! Into send !!!
newvalue is 3
Reset n=5 to n=3

Into while
!!! Out send !!!
newvalue is None
n -= 1

Into while
Into for 2
newvalue is None
n -= 1

Into while
Into for 1
newvalue is None
n -= 1

Into while
Into for 0
newvalue is None
n -= 1



### 1.16. Keeping it Straight

- Generators produce data for iteration


- Coroutines are consumers of data


- To keep your brain from exploding, you don't mix the two concepts together


- Coroutines are not related to iteration


- Note :There is a use of having yield produce a value in a coroutine, but it's not tied to iteration.